# ETL Movies:

### Propuesta de trabajo (requerimientos de aprobación)
Transformaciones: Para este MVP no necesitas perfección, ¡necesitas rapidez! ⏩ Vas a hacer estas, y solo estas, transformaciones a los datos:

* 1 Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

* 2 Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

* 3 Los valores nulos del campo release date deben eliminarse.

* 4 De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

* 5 Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

* 6 Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

## Ingesta de Datasets:

In [99]:
import pandas as pd
df_movies = pd.read_csv('../../Datasets/Movies_datasets/CSVs Movies/movies_dataset_1.csv', encoding='ANSI', sep=',',low_memory=False)

In [100]:
df_movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSO,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,FALSO,7.7,5415.0
1,FALSO,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,FALSO,6.9,2413.0
2,FALSO,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,FALSO,6.5,92.0


In [101]:
columns_to_separate = pd.DataFrame(df_movies['belongs_to_collection'])

In [102]:
columns_to_separate

,belongs_to_collection
0,"{'id': 10194, 'name': 'Toy Story Collection', ..."
1,NaN
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
3,NaN
4,"{'id': 96871, 'name': 'Father of the Bride Col..."
...,...
45461,NaN
45462,NaN
45463,NaN
45464,NaN


In [103]:
print(columns_to_separate[:1])

                               belongs_to_collection
0  {'id': 10194, 'name': 'Toy Story Collection', ...


In [106]:
columns_to_separate.fillna('unknown')

,belongs_to_collection
0,"{'id': 10194, 'name': 'Toy Story Collection', ..."
1,unknown
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
3,unknown
4,"{'id': 96871, 'name': 'Father of the Bride Col..."
...,...
45461,unknown
45462,unknown
45463,unknown
45464,unknown


In [91]:
# Función para extraer los valores de la columna 'belongs_to_collection'
import pandas as pd
import ast

# Supongamos que ya tienes un dataframe llamado 'df' con la columna 'belongs_to_collection'
# df = pd.DataFrame(...)

def extract_collection_info(collection_str):
    if pd.notnull(collection_str):  # Verifica si el valor no es nulo
        try:
            collection_dict = ast.literal_eval(collection_str)  # Convierte la cadena en un diccionario
            return collection_dict['id'], collection_dict['name']
        except (ValueError, TypeError, SyntaxError, KeyError):
            return None, None
    else:
        return None, None



# Aplica la función a la columna 'belongs_to_collection' para obtener dos nuevas columnas
columns_to_separate[['collection_id', 'collection_name']] = columns_to_separate['belongs_to_collection'].apply(extract_collection_info).apply(pd.Series)

# Elimina la columna original 'belongs_to_collection' si ya no la necesitas
columns_to_separate.drop('belongs_to_collection', axis=1, inplace=True)

# Muestra el nuevo dataframe con las columnas separadas
print(columns_to_separate)


       collection_id                 collection_name
0            10194.0            Toy Story Collection
1                NaN                            None
2           119050.0       Grumpy Old Men Collection
3                NaN                            None
4            96871.0  Father of the Bride Collection
...              ...                             ...
45461            NaN                            None
45462            NaN                            None
45463            NaN                            None
45464            NaN                            None
45465            NaN                            None

[45466 rows x 2 columns]


## Debo estudiar cada df movies por separado para acomodarlos dejarlos ok uno por uno y luego concaatenarlos